In [ ]:
%load_ext autoreload
%autoreload 2<

In [ ]:
import pim
import pickle
import numpy as np
import matplotlib.pyplot as plt
import subprocess

from random import uniform
from scipy.optimize import minimize
from pim.models.new.stone import trials, plotter

from loguru import logger

In [ ]:
def descend(T_inbound=1500, T_outbound=1500, noise=0.1, 
            phi=(0.1,1.0),beta=(0.0,1.0),T_half=(0.5,5.0),
            epsilon=(1e1,1e5),l=(1e-4,1e-3),c_tot=(1e-8,0.3)):

    bounds = [phi, beta, T_half, epsilon, l, c_tot]
    initial = np.array([uniform(a,b) for a,b in bounds])

    headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

    with open("path.pickle", "wb") as f:
        pickle.dump((headings.tolist(), velocities.tolist()), f)


    def fastbee(x, *args):

        subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in x], str(noise)])

        with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
            out_path = pickle.load(outbound)
            in_path = pickle.load(inbound)

        v = out_path[1] + in_path[1]

        position = np.zeros(2)
        path = [position]
        for velocity in v:
            position = position + velocity
            path.append(position)

        closest = min(path[T_outbound:], key = np.linalg.norm)
        return np.linalg.norm(closest)

    result = minimize(fastbee, x0=initial, bounds=bounds, tol=1e-7)

    return result

In [ ]:
res = descend(
    phi=(0.001,0.9),
    beta=(0.01,10.0),
    T_half=(0.1,10.0),
    epsilon=(1e3,1e4),
    l=(1e-5,1e-3),
    c_tot=(1e-2,3e-1),
    noise=0.1
)

print(res)

In [ ]:
avg = 0
N = 50

if res:
    for i in range(N):
        headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

        with open("path.pickle", "wb") as f:
            pickle.dump((headings.tolist(), velocities.tolist()), f)

        subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in res.x], str(0.1)])

        with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
            out_path = pickle.load(outbound)
            in_path = pickle.load(inbound)

        v = out_path[1] + in_path[1]

        position = np.zeros(2)
        path = [position]
        for velocity in v:
            position = position + velocity
            path.append(position)

        closest = min(path[1500:], key = np.linalg.norm)
        avg += np.linalg.norm(closest)

    print(avg/N)

In [ ]:
def run_sims(params, N=10, noise=0.1):
    avg = 0
    for i in range(N):
        headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

        with open("path.pickle", "wb") as f:
            pickle.dump((headings.tolist(), velocities.tolist()), f)

        subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in params], str(noise)])

        with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
            out_path = pickle.load(outbound)
            in_path = pickle.load(inbound)

        v = out_path[1] + in_path[1]

        position = np.zeros(2)
        path = [position]
        for velocity in v:
            position = position + velocity
            path.append(position)

        closest = min(path[1500:], key = np.linalg.norm)
        avg += np.linalg.norm(closest)
    return avg/N

def run_slow_sims(params, N=10, noise=0.1):
    avg = 0
    for i in range(N):
        result = pim.models("stone", {
                "model": "stone",
                "T_outbound": 1500,
                "T_inbound": 1500,
                "noise": noise,
                "cx": "dye",
                "phi": params[0],
                "beta": params[1],
                "T_half": params[2],
                "epsilon": params[3],
                "length": params[4],
                "c_tot": params[5]
            }).run("test")

        avg += np.linalg.norm(result.closest_position())
    return avg/N
    

def random_hypersphere_point(n, radius, center):
    #fill a list of n random values
    points = [np.random.uniform(-1.0, 1.0) for r in range(n)]
    #calculate 1 / sqrt of sum of squares
    sqr_red = 1.0 / np.sqrt(sum(i*i for i in points))
    #multiply each point by scale factor 1 / sqrt of sum of squares
    res = np.array(list(map(lambda x: x * sqr_red, points)))
    res *= radius
    res += center
    return res


def random_search(max_iter=10, N=10, phi=(0.01,1.0), beta=(0.001,10.0), T_half=(0.1,10.0),
                epsilon=(1e4,1e5), l=(1e-4,1e-3), c_tot=(0.01,0.3)): # R^n --> R fitness function to be minimized
    # initialize position x in the search space

    # until termination condition is reached:
    #       Sample new pos y from hypersphere of a given radius surrounding point x
    #       if f(y) < f(x), move to new position by setting x = y

    bounds = [phi, beta, T_half, epsilon, l, c_tot]
    x = np.array([np.random.uniform(a,b) for a,b in bounds])
    x = np.array([0.015, 0.1, 137.66577568221356, 2e5, 10e-4, 0.3])
    
    min_dist = 10.1
    i = 0

    while min_dist > 10 and i != max_iter:
        y = random_hypersphere_point(x.size,0.01,x)
        f_y = run_slow_sims(params=y,N=N)
        if f_y < min_dist: 
            x = y
            min_dist = f_y
        i += 1
    
    return [min_dist, x]


In [ ]:
logger.remove()
res = random_search(max_iter=100,N=10)
print(res)

In [ ]:
result = pim.models("stone", {
                "model": "stone",
                "T_outbound": 1500,
                "T_inbound": 1500,
                "noise": 0.1,
                "cx": "dye",
                "phi": res[1][0],
                "beta": res[1][1],
                "T_half": res[1][2],
                "epsilon": res[1][3],
                "length": res[1][4],
                "c_tot": res[1][5]
            }).run("test")
result.report()

In [ ]:
headings, velocities = trials.generate_route(T = 1500, vary_speed = True)

with open("path.pickle", "wb") as f:
    pickle.dump((headings.tolist(), velocities.tolist()), f)

subprocess.run(['../../fastbee/target/release/fastbee', *[str(param) for param in res[1]], str(0.1)])

with open("path.pickle", "rb") as outbound, open("inbound.pickle", "rb") as inbound:
    out_path = pickle.load(outbound)
    in_path = pickle.load(inbound)
    
fig, ax = plotter.plot_route(
    h = out_path[0] + in_path[0],
    v = out_path[1] + in_path[1],
    T_outbound = len(out_path[0]),
    T_inbound = len(in_path[0]),
    plot_speed = True,
    plot_heading = True,
    quiver_color = "black",
    )
plt.show()